In [3]:
import pickle as pkl
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import geopandas as gpd

import numpy as np

import rasterio

import spacy

import gc



nlp = spacy.load('en_core_web_lg')
from sklearn.metrics.pairwise import cosine_similarity

import itertools

# Get Data

## Changes from year to year

In [123]:
changes = pd.read_csv("../../data/compustatChanges_all.csv").drop(columns = ['Unnamed: 0'])

changes.head()

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,gvkey,datadate,year,qtr,companyName,curcdq,assets,costGoodsSold,totalInv,netIncome,...,assetsLast,netIncomeLast,totalRevenueLast,costGoodsSoldLast,totalInvLast,incomeChange,revenueChange,costChange,inventoryChange,assetsPrev
0,1004,20010228,2000,3.0,AAR CORP,USD,754.718,159.537,354.479,5.388,...,753.755,10.955,272.331,222.672,318.933,-0.508170,-0.265339,-0.283534,0.111453,753.755
1,1004,20010531,2000,4.0,AAR CORP,USD,701.854,180.131,320.590,5.706,...,740.998,2.471,225.079,182.440,336.018,1.309187,-0.017772,-0.012656,-0.045914,740.998
2,1004,20010831,2001,1.0,AAR CORP,USD,758.503,168.829,340.683,0.486,...,747.543,3.159,241.770,202.661,343.815,-0.846154,-0.160388,-0.166939,-0.009110,747.543
3,1004,20011130,2001,2.0,AAR CORP,USD,714.208,118.697,265.818,-54.484,...,772.941,4.278,211.335,171.482,344.778,-13.735858,-0.314411,-0.307817,-0.229017,772.941
4,1004,20020228,2001,3.0,AAR CORP,USD,690.681,118.267,271.342,-2.290,...,754.718,5.388,200.071,159.537,354.479,-1.425019,-0.282970,-0.258686,-0.234533,754.718


In [124]:
otherControls = pd.read_csv('../../data/companyData/otherControls.csv').\
    drop(columns = {'Unnamed: 0', 'fyearq'}).rename(columns = {'year_toMatchOn': 'year',
                                                              'fqtr': 'qtr'})

otherControls.head()

,gvkey,qtr,assetsLagged,netIncomeLagged,roa_lagged,famafrench,year,earliestYear,ageTercile,sizeTercile,profitTercile
0,1010,1.0,3422.936770,46.757990,0.013660,26.0,1998,1962,0,2.0,2.0
1,1010,2.0,3454.132166,18.500640,0.005356,26.0,1998,1962,0,2.0,1.0
2,1010,3.0,3487.709411,10.470317,0.003002,26.0,1998,1962,0,2.0,1.0
3,1010,4.0,3448.667709,68.705992,0.019922,26.0,1998,1962,0,2.0,2.0
4,1010,1.0,4322.135119,260.987674,0.060384,26.0,1999,1962,0,2.0,2.0


In [125]:
otherControls.head()

,gvkey,qtr,assetsLagged,netIncomeLagged,roa_lagged,famafrench,year,earliestYear,ageTercile,sizeTercile,profitTercile
0,1010,1.0,3422.936770,46.757990,0.013660,26.0,1998,1962,0,2.0,2.0
1,1010,2.0,3454.132166,18.500640,0.005356,26.0,1998,1962,0,2.0,1.0
2,1010,3.0,3487.709411,10.470317,0.003002,26.0,1998,1962,0,2.0,1.0
3,1010,4.0,3448.667709,68.705992,0.019922,26.0,1998,1962,0,2.0,2.0
4,1010,1.0,4322.135119,260.987674,0.060384,26.0,1999,1962,0,2.0,2.0


In [126]:
print(changes.shape)
changes = changes.merge(otherControls)
print(changes.shape)


industries = changes[['gvkey','famafrench']].drop_duplicates()

(890758, 29)
(829560, 37)


In [127]:
industries.to_csv("../../data/companyData/gvkeyIndustries.csv")

In [ ]:
'''changes.to_csv("../../data/companyData/compustatChanges_withControls.csv")
changes.head()'''

In [ ]:
changes = pd.read_csv("../../data/companyData/compustatChanges_withControls.csv").drop(columns = {'Unnamed: 0',
                                                                                                 'Unnamed: 0.1',
                                                                                                 'Unnamed: 0.1.1'})
changes.head()

Put in the calendar quarters and fiscal quarter data.

In [ ]:
quarters = pd.read_csv("../../data/companyData/fiscalYears.csv")
quarters.head()

In [ ]:
len(quarters.gvkey.unique())

In [ ]:
sum((quarters.fyr == 12) | 
   (quarters.fyr == 3) | 
   (quarters.fyr == 6) | 
   (quarters.fyr == 9))/quarters.shape[0]

In [ ]:
quarters = quarters[(quarters.fyr == 12) | 
   (quarters.fyr == 3) | 
   (quarters.fyr == 6) | 
   (quarters.fyr == 9)][['gvkey','datadate','datacqtr','datafqtr','fyr']].reset_index(drop = True)


In [ ]:
quarters.head()

Merge the quarter data into the change data, and make sure that the quarters that are used line up with the calendar quarters.

In [ ]:
changesCal = changes[changes.gvkey.isin(quarters.gvkey.unique())]

changesCal = changesCal.merge(quarters)

print(changesCal.shape[0]/changes.shape[0])

In [ ]:
changesCal.loc[~(changesCal.datacqtr.isna()), 'year'] = changesCal.datacqtr.str.slice(0,4)
changesCal.loc[~(changesCal.datacqtr.isna()), 'qtr']  = changesCal.datacqtr.str.slice(5,6)

changesCal['DATE'] = pd.to_datetime(changesCal['datadate'])

changesCal.loc[(changesCal.datacqtr.isna()), 'year'] = changesCal.DATE.dt.year
changesCal.loc[(changesCal.datacqtr.isna()), 'qtr']  = changesCal.DATE.dt.quarter

changesCal['year'] = changesCal.year.astype('int64')
changesCal['qtr']  = changesCal.qtr.astype('int64')

print(changesCal.shape,changesCal.head())

In [ ]:
changesCal.to_csv("../../data/companyData/compustatChanges_withControls.csv")
changesCal.head()

In [ ]:
changesCal = pd.read_csv("../../data/companyData/compustatChanges_withControls.csv")
changesCal.head()

# Compustat and ABI Linking

In [136]:
gvKey_abiLinkingTable = pd.read_csv('../../data/companyData/linkingTable.csv').drop(columns = ['Unnamed: 0'])

base_columns = gvKey_abiLinkingTable.columns 
customer_columns = "customer_" + base_columns
supplier_columns = "supplier_" + base_columns



hasMatch = gvKey_abiLinkingTable.gvkey.unique()

gvKey_abiLinkingTable.head()


,cstatCompanies,igCompanies,delete,gvkey,abi
0,asa gold and precious metals,asa gold precious metals,NaN,1062,402180222
1,adams diversified equity fd,adams diversified equity fund,NaN,1119,397759739
2,allen organ,allen organ,NaN,1283,400700704
3,american physicians svc gp,american physicians svc,NaN,1539,218548014
4,american science engineering,american science engineering,NaN,1554,441435880


---------------------------------

# Get all change data together
Get the linking table and merge the abi labels into the change df. 

Then, merge the location data into the change data and get as complete a record of companies as possible given the HQ data.

In [ ]:
gvKey_abiLinkingTable = pd.read_csv('../../data/companyData/linkingTable.csv').drop(columns = ['Unnamed: 0'])

changes = pd.read_csv("../../data/companyData/compustatChanges_withControls.csv").drop(columns = ['Unnamed: 0'])
print(changes.shape, changes.head())


changesABI = changes.merge(gvKey_abiLinkingTable, on ='gvkey').drop(columns = {'state','city'})
print(changesABI.shape, changesABI.head())

Now merge in the hq information.

In [ ]:
canadian = ['ON', 'AB','QC', 'BC', 'NS', 'NF', 'SK', 'MB', 'NB']
changes = changes[~(changes.state.isin(canadian)) & ~changes.state.isna()]

changes['addzip'] = changes.addzip.astype('str').str.slice(0,5)

changes.state.unique()

In [ ]:
hq = pd.read_csv("../../data/ig_uniqueHQs_multLocations.csv").\
    drop(columns = {'Unnamed: 0'}).\
    rename(columns = {'archive_version_year': 'year'})

hq['year'] = hq.year.astype('int64')

igChanges = changesABI.merge(hq)
print(igChanges.shape, igChanges.head())


hq.head()

In [ ]:
igChanges.to_csv("../../data/companyData/igData.csv")

At this point, we have zip information in the following forms (from most to least examples):
    - changes: all compustat companies, from the compustat address system
    - igChanges: subset of compustat companies, from the ig merge
    - subset of compustat companies that have SC information and survived the ig merge
    
We could potentially look at the subset of compustat companies for which we have SC information, usign the compustat address system as well.

For now: follow similar trajectory as before but add in weather data for all cstat companies and all ig-merged companies.

First: pull all zips that are mentioned in changes and igChanges and use this to get the weather data.



In [ ]:
changes = changes[(~changes.addzip.isna()) & (changes.addzip != 'nan')]
relevantZips = changes.addzip.astype('int64').append(igChanges.zipcode).unique()

changes.rename(columns = {'addzip': 'zipcode'}, inplace = True)
changes.drop(columns = {'cik',
     'datadate','costat', 'add1', 'add2', 'city', 'sic', 'state'}, inplace = True)

In [ ]:
len(relevantZips)

In [ ]:
# relevantZips = allCustomerData.zipcode.append(allSupplierData.zipcode).unique()
outfile =  '../../data/companyData/relevantZips.pkl'
with open(outfile, 'wb') as pickle_file:
    pkl.dump(relevantZips, pickle_file)

------------------------------------------------

# Stocks

In [ ]:
igChanges = pd.read_csv("../../data/companyData/igData.csv").\
    drop(columns = {'Unnamed: 0'})
igChanges.head()

In [ ]:
with open('../../data/stockReturns.pkl', 'rb') as f:
    stocks = pkl.load(f)[['date','gvkey','RET']]

In [ ]:
stocks.head()

In [ ]:
sum(stocks.gvkey.isna())

In [ ]:
stocks = stocks[stocks.date.dt.year > 2008]

stocks['qtr']  = stocks.date.dt.quarter
stocks['year'] = stocks.date.dt.year

stocks = stocks[~stocks.gvkey.isna()]
stocks['gvkey'] = stocks['gvkey'].astype(int)
stocks.shape

In [ ]:
igChanges.columns

In [ ]:
companyControls = igChanges[['gvkey','year','qtr','famafrench','ageTercile','sizeTercile','profitTercile','zipcode']]
companyControls.head()

In [ ]:
print(stocks.dtypes, companyControls.dtypes)

In [ ]:
stocksWithControls = stocks.merge(companyControls)
print(stocksWithControls.shape,stocks.shape,companyControls.shape)
stocksWithControls.head()

In [ ]:
del stocks
del companyControls
del igChanges
gc.collect()

In [ ]:
annualWeather = pd.read_csv("../../data/companyData/stockWeather_annual.csv").\
    drop(columns = {'Unnamed: 0'})

annualWeather = annualWeather[~annualWeather.temp_annualLast5.isna()].reset_index(drop = True)

annualWeather['date'] = pd.to_datetime(annualWeather['date'],
                                   format = "%Y%m%d")

annualWeather.rename(columns = {'ZIP': 'zipcode'}, inplace = True)
print(annualWeather.dtypes)
annualWeather.head()

In [ ]:
allWeather = pd.read_csv("../../data/companyData/stockWeather_zipQuarterQuants.csv").\
    drop(columns = {'Unnamed: 0'})

allWeather = allWeather[~allWeather.temp_zipQuarterLast5.isna()].reset_index(drop = True)

allWeather['date'] = pd.to_datetime(allWeather['date'],
                                   format = "%Y-%m-%d")

allWeather.rename(columns = {'ZIP': 'zipcode'}, inplace = True)
print(allWeather.dtypes)
allWeather.head()

In [ ]:
stocksWithControlsWeather = stocksWithControls.merge(allWeather).merge(annualWeather)
print(stocksWithControlsWeather.shape,allWeather.shape)

stocksWithControlsWeather.head()

In [ ]:
stocksWithControlsWeather.to_csv("../../data/2+32+100+600sfgcompanyData/stocksWithControlsWeather.csv")

In [ ]:
sum(stocksWithControlsWeather.RET.isna())

In [ ]:
stocksWithControlsWeather = pd.read_csv("../../data/companyData/stocksWithControlsWeather.csv").drop(columns = {'Unnamed: 0'})
stocksWithControlsWeather.head()

In [ ]:
sum(stocksWithControlsWeather.gvkey.isna())

--------------------

# Weather Data
First do this on the HQ zipcodes.

In [ ]:
allWeather = pd.read_csv("../../data/companyData/revised_allWeatherBins_2009to2019.csv").\
    drop(columns = {"Unnamed: 0"})

allWeather['yearQtr'] = allWeather.year + (allWeather.qtr - 1)/4

col = allWeather.pop("yearQtr")
allWeather.insert(0, col.name, col)

lag1 = allWeather.copy()
lag1['yearQtr'] += 0.25
for colname in lag1.columns[4:]:
    lag1.rename(columns = {colname: 'lag1_' + colname}, inplace = True)
lag1.drop(columns = {'year','qtr'},inplace = True)

    
lag2 = allWeather.copy()
lag2['yearQtr'] += 0.5
for colname in lag2.columns[4:]:
    lag2.rename(columns = {colname: 'lag2_' + colname}, inplace = True)
lag2.drop(columns = {'year','qtr'},inplace = True)


lag3 = allWeather.copy()
lag3['yearQtr'] += 0.75
for colname in lag3.columns[4:]:
    lag3.rename(columns = {colname: 'lag3_' + colname}, inplace = True)
lag3.drop(columns = {'year','qtr'},inplace = True)


lag4 = allWeather.copy()
lag4['yearQtr'] += 1
for colname in lag4.columns[4:]:
    lag4.rename(columns = {colname: 'lag4_' + colname}, inplace = True)
lag4.drop(columns = {'year','qtr'},inplace = True)


print(allWeather.shape)

allWeather_withLags = allWeather.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

print(allWeather_withLags.year.value_counts())

allWeather_withLags.to_csv("../../data/companyData/allWeather_withLags.csv")

Do this across all zips, for the establishment records. We'll put this into a different format right after, and then change the columns and whatnot.

In [ ]:
allWeather = pd.read_csv("../../data/companyData/revised_allWeatherBins_2009to2019_allZips.csv").\
    drop(columns = {"Unnamed: 0", 'Unnamed: 0.1'})

allWeather['yearQtr'] = allWeather.year + (allWeather.qtr - 1)/4

col = allWeather.pop("yearQtr")
allWeather.insert(0, col.name, col)

lag1 = allWeather.copy()
lag1['yearQtr'] += 0.25
for colname in lag1.columns[4:]:
    lag1.rename(columns = {colname: 'lag1_' + colname}, inplace = True)
lag1.drop(columns = {'year','qtr'},inplace = True)

    
lag2 = allWeather.copy()
lag2['yearQtr'] += 0.5
for colname in lag2.columns[4:]:
    lag2.rename(columns = {colname: 'lag2_' + colname}, inplace = True)
lag2.drop(columns = {'year','qtr'},inplace = True)


lag3 = allWeather.copy()
lag3['yearQtr'] += 0.75
for colname in lag3.columns[4:]:
    lag3.rename(columns = {colname: 'lag3_' + colname}, inplace = True)
lag3.drop(columns = {'year','qtr'},inplace = True)


lag4 = allWeather.copy()
lag4['yearQtr'] += 1
for colname in lag4.columns[4:]:
    lag4.rename(columns = {colname: 'lag4_' + colname}, inplace = True)
lag4.drop(columns = {'year','qtr'},inplace = True)


print(allWeather.shape)

allWeather_withLags = allWeather.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

print(allWeather_withLags.year.value_counts())

allWeather_withLags.to_csv("../../data/companyData/allWeather_withLags_allZips.csv")

Now do the same for the industry-specific weather.

In [ ]:
# allWeather = pd.read_csv("../../../../../../../Volumes/backup2/dissData/prism/allWeatherBins_2010.2019.csv").\
allWeather_byInd = pd.read_csv("../../data/companyData/revised_allWeatherBins_2009to2019_byInd.csv").\
    drop(columns = {"Unnamed: 0"})
'''[['famafrench','zipcode','yearQuarter', 
                                    'temp_ffquant_0.95','temp_indQuarterquant_0.95',
                                   'temp5Days_ffquant_0.95', 'temp5Days_indQuarterquant_0.95',
                                   'precip_ffquant_0.95', 'precip_indQuarterquant_0.95',
                                   'precip5Days_ffquant_0.95', 'precip5Days_indQuarterquant_0.95']]
'''
allWeather_byInd['year'] = allWeather_byInd.yearQuarter.str.slice(0,4).astype('int64')
allWeather_byInd['qtr']  = allWeather_byInd.yearQuarter.str.slice(5,6).astype('int64')
allWeather_byInd['yearQtr'] = allWeather_byInd.year + (allWeather_byInd.qtr - 1)/4

allWeather_byInd = allWeather_byInd.astype({'year':       'category',
                         'qtr':        'category',
                         'zipcode':    'category',
                         'famafrench': 'category'})

changes['zipcode'] = changes['zipcode'].astype({'zipcode': 'int64'})

changes = changes.astype({'year':       'category',
                          'qtr':        'category',
                          'zipcode':    'category',
                          'famafrench': 'category'})

col = allWeather_byInd.pop("year")
allWeather_byInd.insert(0, col.name, col)

col = allWeather_byInd.pop("qtr")
allWeather_byInd.insert(0, col.name, col)


col = allWeather_byInd.pop("yearQtr")
allWeather_byInd.insert(0, col.name, col)

allWeather_byInd.drop(columns = {'yearQuarter'}, inplace = True)

print(allWeather_byInd.head())

In [ ]:
lag1 = allWeather_byInd.copy()
lag1['yearQtr'] += 0.25
for colname in lag1.columns[5:]:
    lag1.rename(columns = {colname: 'lag1_' + colname}, inplace = True)
lag1.drop(columns = {'year','qtr'},inplace = True)
lag1 = lag1.astype({'yearQtr':       'category'})

    
lag2 = allWeather_byInd.copy()
lag2['yearQtr'] += 0.5
for colname in lag2.columns[5:]:
    lag2.rename(columns = {colname: 'lag2_' + colname}, inplace = True)
lag2.drop(columns = {'year','qtr'},inplace = True)
lag2 = lag2.astype({'yearQtr':       'category'})


lag3 = allWeather_byInd.copy()
lag3['yearQtr'] += 0.75
for colname in lag3.columns[5:]:
    lag3.rename(columns = {colname: 'lag3_' + colname}, inplace = True)
lag3.drop(columns = {'year','qtr'},inplace = True)
lag3 = lag3.astype({'yearQtr':       'category'})


lag4 = allWeather_byInd.copy()
lag4['yearQtr'] += 1
for colname in lag4.columns[5:]:
    lag4.rename(columns = {colname: 'lag4_' + colname}, inplace = True)
lag4.drop(columns = {'year','qtr'},inplace = True)
lag4 = lag4.astype({'yearQtr':       'category'})


allWeather_byInd = allWeather_byInd.astype({'yearQtr':       'category'})


print(allWeather_byInd.shape)


allWeather_byInd.head()


'''allWeather_byInd_withLags = allWeather_byInd.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

allWeather_byInd_withLags.year.value_counts()
'''

In [ ]:
allWeather_byInd_withLags = allWeather_byInd.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

In [ ]:
allWeather_byInd_withLags.shape

In [ ]:
allWeather_byInd_withLags.to_csv("../../data/companyData/allWeather_byInd_withLags.csv")

In [ ]:
del allWeather_byInd
del lag1
del lag2
del lag3
del lag4
gc.collect()

In [ ]:
allWeather_byInd_withLags = pd.read_csv("../../data/companyData/allWeather_byInd_withLags.csv")
allWeather_byInd_withLags.head()

# Locations
Create a separate definition of weather based not on HQ but on employee-weighted establishment footprint.

In [ ]:
fractions = pd.read_csv('../../data/companyData/fractionEmployees_byEstablishment.csv').\
    drop(columns = {"Unnamed: 0", 'latitude','longitude'}).rename(columns = {'archive_version_year': 'year',
                                                    'parent_number': 'abi'})

fractions['year']    = fractions.year.astype('int64')
fractions['zipcode'] = fractions.zipcode.astype('int64')
fractions.head()

gvKey_abiLinkingTable = pd.read_csv('../../data/companyData/linkingTable.csv').drop(columns = ['Unnamed: 0'])

print(gvKey_abiLinkingTable.abi)

gvKey_abiLinkingTable.head()

fractions = fractions[['year','abi','zipcode','locationFracOfEmployees']].merge(gvKey_abiLinkingTable[['abi','gvkey']])

fractions = fractions.astype({'year':       'category',
                           'zipcode':    'category'})

fractions.head()

In [ ]:
fractionsWithWeather = fractions.merge(allWeather_withLags_allZips) 
fractionsWithWeather.drop(columns = {'abi','zipcode'}, inplace = True)

print(fractionsWithWeather.shape)
fractionsWithWeather.head()

In [ ]:
fractionsWithWeather[fractionsWithWeather.gvkey == 1004]

In [ ]:
del allWeather_withLags
del fractions
del gvKey_abiLinkingTable
gc.collect()

In [ ]:
for col in fractionsWithWeather.columns[4:]:
    fractionsWithWeather[col] = fractionsWithWeather[col] * fractionsWithWeather.locationFracOfEmployees

In [ ]:
g = fractionsWithWeather.groupby(['gvkey','year','qtr']).sum().reset_index()
g.drop(columns = {'locationFracOfEmployees'}, inplace = True)

for colname in g.columns[3:]:
    g.rename(columns = {colname: 'empWt_' + colname}, inplace = True)

g.head()

In [ ]:
g.to_csv("../../data/companyData/weatherByEstablishment.csv")

In [ ]:
establishmentZips = fractions.zipcode.unique()
len(establishmentZips)

## create the original weather with lags dataset

In [ ]:
allWeather_withLags = pd.read_csv("../../data/companyData/allWeather_withLags.csv").\
    drop(columns = {"Unnamed: 0", 'yearQtr'})

averages = pd.read_csv("../../data/companyData/quarterlyStatsByZip.csv").\
    drop(columns = {"Unnamed: 0"}).rename(columns = {'ZIP': 'zipcode'})


averages['qtr'] = averages.quarter.str.slice(1,2).astype('float')
averages.drop(columns = {'quarter'}, inplace = True) 

print(len(averages.zipcode.unique()))

averages.head()

allWeather_withLags       = allWeather_withLags.astype({'year':       'category',
                           'qtr':        'category',
                           'zipcode':    'category'})
averages                  = averages.astype({'qtr':        'category',
                           'zipcode':    'category'})

allWeather_byInd_withLags = pd.read_csv("../../data/companyData/allWeather_byInd_withLags.csv").\
    drop(columns = {"Unnamed: 0", 'yearQtr'})
allWeather_byInd_withLags = allWeather_byInd_withLags.astype({'year':       'category',
                           'qtr':        'category',
                           'zipcode':    'category'})

Create direct effects database. Merge weather to full cstat database.

In [ ]:
allWeather_withLags.zipcode

In [ ]:
changes.zipcode = changes.zipcode.astype('int64')

In [ ]:
# changes['zipcode']  = changes['zipcode'].astype('int64')
changesWithWeather = changes.merge(allWeather_withLags).merge(allWeather_byInd_withLags).merge(averages).merge(g)
print(changes.shape,changesWithWeather.shape)

In [ ]:
changesWithWeather.to_csv("../../data/companyData/cstatWithWeather.csv")

Merge weather to the ig-cstat database.

In [ ]:
igChangesWithWeather = igChanges.merge(allWeather_withLags).merge(allWeather_byInd_withLags).merge(averages).merge(g)
igChangesWithWeather.shape

In [ ]:
igChangesWithWeather.to_csv("../../data/companyData/igWithWeather.csv")

In [ ]:
igChangesWithWeather.head()

In [ ]:
igChangesWithWeather['temp_zipquant_0.95'].describe()

In [ ]:
igChangesWithWeather['temp5Days_ffquant_0.95'].describe()

# Indirect
Introduce the SC Data.

In [ ]:
# this does a little bit of a test on the reporting requirements. 
# number 

'''c_linksTest = pd.read_csv("../../data/companyData/compustatSCLinked.csv")[['srcdate','gvkey','cgvkey']]
c_linksTest['year'] = c_linksTest.srcdate.astype('str').str.slice(0,4).astype('int64')

bs = c_linksTest[c_linksTest.year < 2014]
print("Customers per supplier, 1978-2013 Pd: ", len(bs.cgvkey.unique())/len(bs.gvkey.unique()))

bs2 = c_linksTest[c_linksTest.year > 2010]
print("Customers per supplier, Recent Pd: ", len(bs2.cgvkey.unique())/len(bs2.gvkey.unique()))'''


In [162]:
c_links = pd.read_csv("../../data/companyData/compustatSCLinked.csv") # pd.read_csv("../../data/companyData/compustatSCLinked.csv")

c_links['year'] = c_links.srcdate.astype('str').str.slice(0,4).astype('int64')

c_links = c_links[c_links.year > 1999][['year','gvkey','cgvkey','salecs']].\
    rename(columns = {'cgvkey': 'customer_gvkey','gvkey': 'supplier_gvkey'})

c_links['year'] = pd.to_datetime(c_links.year, format = '%Y')


c_links.head()

,year,supplier_gvkey,customer_gvkey,salecs
70,2002-01-01,1013,2136,111.056
71,2004-01-01,1013,2136,104.312
72,2005-01-01,1013,2136,146.000
73,2006-01-01,1013,2136,205.000
74,2007-01-01,1013,2136,236.000


In [163]:
supplierCombos = c_links[['supplier_gvkey', 'customer_gvkey']].drop_duplicates().reset_index(drop = True)

print(supplierCombos.shape)

supplierCombos.head()

(16812, 2)


,supplier_gvkey,customer_gvkey
0,1013,2136
1,1013,9899
2,1021,61494
3,1021,25880
4,1048,11552


We'll follow Barrot Sauvagnat in assuming that a supplier relationship holds for every year between the first and last year in which a customer is reported. This is going to take a little bit of work. We'll try it like this: 
- subset dataframe to a specific supplier-customer pair
- fill in data for every year that's missing

Then, apply this row-wise to all rows of the unique supplierCombos df above using: https://stackoverflow.com/questions/61942138/apply-function-row-wise-to-pandas-dataframe

In [164]:
def fillYear(supplier, customer, scData = c_links):
    c_linksTemp = scData[(scData.supplier_gvkey == supplier) & \
                      (scData.customer_gvkey == customer)].reset_index(drop = True)
    
    # if there are na values and non-na values for the same supplier-cust combination, then 
    # select for only the non-na values, by (1) replacing na with negative, (2) 1
    c_linksTemp['salecs'] = c_linksTemp['salecs'].fillna(-5)
    c_linksTemp = c_linksTemp.loc[c_linksTemp.reset_index().groupby(['year','supplier_gvkey', 'customer_gvkey'])['salecs'].idxmax()]

    
    # now: find the start and end of the data series
    first = c_linksTemp.year.min()
    last  = c_linksTemp.year.max()

    c_linksTemp = c_linksTemp.set_index('year') 

    c_linksTemp = c_linksTemp.reindex(pd.date_range(first, last, freq = 'YS')).\
        reset_index().rename(columns = {'index': 'year'})

    # and impute all values within the series
    c_linksTemp = c_linksTemp.groupby(c_linksTemp.year.dt.time).ffill()
    
    return(c_linksTemp)

In [165]:
fillYear(supplierCombos.supplier_gvkey[0], supplierCombos.customer_gvkey[0])

,year,supplier_gvkey,customer_gvkey,salecs
0,2002-01-01,1013.0,2136.0,111.056
1,2003-01-01,1013.0,2136.0,111.056
2,2004-01-01,1013.0,2136.0,104.312
3,2005-01-01,1013.0,2136.0,146.000
4,2006-01-01,1013.0,2136.0,205.000
5,2007-01-01,1013.0,2136.0,236.000
6,2008-01-01,1013.0,2136.0,240.000
7,2009-01-01,1013.0,2136.0,176.000
8,2010-01-01,1013.0,2136.0,146.000


In [166]:
start = time.time()
print(c_links.shape)
c_linksImpd_list = supplierCombos.apply(lambda row: fillYear(row['supplier_gvkey'], row['customer_gvkey']), axis = 1)
c_linksImpd_df   = pd.concat(list(c_linksImpd_list))
print(c_linksImpd_df.shape)
print(time.time() - start)

c_linksImpd_df['year'] = c_linksImpd_df.year.dt.year

(65270, 4)
(68771, 4)
70.1522970199585


In [167]:
c_linksImpd_df.loc[c_linksImpd_df.salecs == -5, 'salecs'] = float('nan')


In [173]:
c_linksImpd_df.salecs

0    111.056
1    111.056
2    104.312
3    146.000
4    205.000
      ...   
1    179.284
0     34.418
0     25.334
0    283.318
1    316.116
Name: salecs, Length: 68771, dtype: float64

In [174]:
industries = pd.read_csv("../../data/companyData/gvkeyIndustries.csv").drop(columns = {'Unnamed: 0'})
print(industries)

        gvkey  famafrench
0        1004        42.0
1        1010        26.0
2        1013        37.0
3        1019        35.0
4        1021        12.0
...       ...         ...
25237  345920        21.0
25238  345980        44.0
25239  347085        35.0
25240  351491        24.0
25241  351590        24.0

[25242 rows x 2 columns]


In [175]:
c_links = c_linksImpd_df.copy()

print(c_links.shape)

print(c_links.head())

industries.columns = ['customer_gvkey','customer_famafrench']

c_links = c_links.merge(industries)

industries.columns = ['supplier_gvkey','supplier_famafrench']

c_links = c_links.merge(industries)
print(c_links.head(), c_links.shape)


c_links.to_csv("../../data/companyData/c_links.csv")


(68771, 4)
   year  supplier_gvkey  customer_gvkey   salecs
0  2002          1013.0          2136.0  111.056
1  2003          1013.0          2136.0  111.056
2  2004          1013.0          2136.0  104.312
3  2005          1013.0          2136.0  146.000
4  2006          1013.0          2136.0  205.000
   year  supplier_gvkey  customer_gvkey   salecs  customer_famafrench  \
0  2002          1013.0          2136.0  111.056                 33.0   
1  2003          1013.0          2136.0  111.056                 33.0   
2  2004          1013.0          2136.0  104.312                 33.0   
3  2005          1013.0          2136.0  146.000                 33.0   
4  2006          1013.0          2136.0  205.000                 33.0   

   supplier_famafrench  
0                 37.0  
1                 37.0  
2                 37.0  
3                 37.0  
4                 37.0   (66561, 6)


Now see if it's common to have one in and one out of the industries of interest. 

For now, let's keep all the different industry types.

We can always filter later if we need to.

In [176]:
#########################
# merge in customer information
gvKey_abiLinkingTable.columns = customer_columns

print(c_links.shape)
c_linksMerge1 = c_links.merge(gvKey_abiLinkingTable, on ='customer_gvkey')
print(c_links.shape,c_linksMerge1.shape)



#########################
# and merge in supplier 
gvKey_abiLinkingTable.columns = supplier_columns

print(c_links.shape)
c_linksMerge2 = c_linksMerge1.merge(gvKey_abiLinkingTable, on ='supplier_gvkey')
print(c_links.shape,c_linksMerge2.shape)

c_linksMerge2.to_csv("../../data/companyData/clinks_IG_selected.csv")

(66561, 6)
(66561, 6) (47330, 10)
(66561, 6)
(66561, 6) (35055, 14)


This is probably because: (1) companies are not in North America, or (2) companies are not in the physical goods industries we're interested in. We can verify this though: look at c_links where both the customer and supplier are in the dataset of interest.

In [177]:
chq     = pd.read_csv("../../data/chq.csv",dtype={'cstatZipcode': 'object'}).drop(columns = {'Unnamed: 0'})

c_linkTest = c_links[c_links.customer_gvkey.isin(chq.gvkey.unique()) & \
                     c_links.supplier_gvkey.isin(chq.gvkey.unique())]

print("Percent of firms with a match: ", c_linksMerge2.shape[0]/c_linkTest.shape[0])

Percent of firms with a match:  0.7116466026512922


It's entirely possible that we have too small of a sample from the 2010s alone. Let's just try it though and see how it goes.

First, make a sample with the companies on three years of either side of when it reports another customer.

In [178]:
def makeOneEitherSide(df): 
    yrPlus1 = df.copy(); yrPlus1['year'] += 1
    # yrPlus2 = df.copy(); yrPlus2['year'] += 1
    # yrPlus3 = df.copy(); yrPlus3['year'] += 1
    
    yrMinus1 = df.copy(); yrMinus1['year'] -= 1
    # yrMinus2 = df.copy(); yrMinus2['year'] -= 1
    # yrMinus3 = df.copy(); yrMinus3['year'] -= 1
    
    all = pd.concat([yrPlus1,yrMinus1]) # pd.concat([yrPlus1,yrPlus2,yrPlus3,yrMinus1,yrMinus2,yrMinus3])
    
    return(all)

In [179]:
scTableCustomers = c_linksMerge2.copy()[['year','customer_gvkey','customer_abi']].drop_duplicates()
scTableSuppliers = c_linksMerge2.copy()[['year','supplier_gvkey','supplier_abi']].drop_duplicates()

allCustomerData = makeOneEitherSide(scTableCustomers)
allCustomerData.columns = ['year','gvkey','abi']


allSupplierData = makeOneEitherSide(scTableSuppliers)
allSupplierData.columns = ['year','gvkey','abi']

allAbi = allCustomerData.abi.append(allSupplierData.abi).drop_duplicates()

hqsOnly = pd.read_csv("../../data/ig_uniqueHQs.csv").drop(columns = {'Unnamed: 0'})

hq = pd.read_csv("../../data/ig_uniqueHQs_multLocations.csv").\
    drop(columns = {'Unnamed: 0'}).\
    rename(columns = {'archive_version_year': 'year'})

hq['year'] = hq.year.astype('int64')

hqRelevant = hq[hq.abi.isin(allAbi)]


allSupplierData = allSupplierData.merge(hqRelevant).drop_duplicates()
allCustomerData = allCustomerData.merge(hqRelevant).drop_duplicates()

print(allSupplierData.head())

allCustomerData.to_csv("../../data/companyData/allCustomerData.csv")
allSupplierData.to_csv("../../data/companyData/allSupplierData.csv")

   year   gvkey      abi ticker                     company state  \
0  2003  1013.0  7523129    NaN  ADC TELECOMMUNICATIONS INC    MN   
2  2004  1013.0  7523129    NaN  ADC TELECOMMUNICATIONS INC    MN   
4  2005  1013.0  7523129    NaN  ADC TELECOMMUNICATIONS INC    MN   
6  2006  1013.0  7523129    NaN  ADC TELECOMMUNICATIONS INC    MN   
8  2007  1013.0  7523129    NaN  ADC TELECOMMUNICATIONS INC    MN   

           city       address_line_1  zipcode  latitude  longitude  \
0  EDEN PRAIRIE  13625 TECHNOLOGY DR    55344  44.85645  -93.45199   
2  EDEN PRAIRIE  13625 TECHNOLOGY DR    55344  44.85645  -93.45199   
4  EDEN PRAIRIE  13625 TECHNOLOGY DR    55344  44.85645  -93.45199   
6  EDEN PRAIRIE  13625 TECHNOLOGY DR    55344  44.85645  -93.45199   
8  EDEN PRAIRIE  13625 TECHNOLOGY DR    55344  44.85645  -93.45199   

   parent_employee_size_code  location_employee_size_code  employeesAtLocation  
0                      250.0                        250.0                  1.0  
2 

## Find Customer and Supplier pairings and merge with change data
### Can pick up here

In [228]:
allSupplierData = pd.read_csv("../../data/companyData/allSupplierData.csv").drop(columns = ['Unnamed: 0'])
allCustomerData = pd.read_csv("../../data/companyData/allCustomerData.csv").drop(columns = ['Unnamed: 0'])

changes = pd.read_csv("../../data/companyData/compustatChanges_withControls.csv").drop(columns = ['Unnamed: 0'])
print(changes.columns)
suppliers = changes.merge(allSupplierData[['year','gvkey','zipcode','employeesAtLocation']])
print(suppliers.shape)

customers = changes.merge(allCustomerData[['year','gvkey','zipcode','employeesAtLocation']])
print(customers.head())

print(allCustomerData.shape,allSupplierData.shape)

Index(['gvkey', 'datadate', 'year', 'qtr', 'companyName', 'curcdq', 'assets',
       'costGoodsSold', 'totalInv', 'netIncome', 'totalRevenue', 'cik',
       'costat', 'add1', 'add2', 'addzip', 'city', 'sic', 'state',
       'assetsLast', 'netIncomeLast', 'totalRevenueLast', 'costGoodsSoldLast',
       'totalInvLast', 'incomeChange', 'revenueChange', 'costChange',
       'inventoryChange', 'assetsPrev', 'assetsLagged', 'netIncomeLagged',
       'roa_lagged', 'famafrench', 'earliestYear', 'ageTercile', 'sizeTercile',
       'profitTercile', 'datacqtr', 'datafqtr', 'fyr', 'DATE'],
      dtype='object')
(51385, 43)
   gvkey  datadate  year  qtr                  companyName curcdq   assets  \
0   1045  20030331  2003    1  AMERICAN AIRLINES GROUP INC    USD  29086.0   
1   1045  20030630  2003    2  AMERICAN AIRLINES GROUP INC    USD  29220.0   
2   1045  20030930  2003    3  AMERICAN AIRLINES GROUP INC    USD  29943.0   
3   1045  20031231  2003    4  AMERICAN AIRLINES GROUP INC    USD  29

## Get first-hop SC data

In [181]:
c_links = pd.read_csv("../../data/companyData/clinks_IG_selected.csv").drop(columns = {'Unnamed: 0'})
c_links.head()

,year,supplier_gvkey,customer_gvkey,salecs,customer_famafrench,supplier_famafrench,customer_cstatCompanies,customer_igCompanies,customer_delete,customer_abi,supplier_cstatCompanies,supplier_igCompanies,supplier_delete,supplier_abi
0,2002,1013.0,2136.0,111.056,33.0,37.0,verizonmmunications,verizonmmunications,NaN,7564776,adc telecommunications,adc telecommunications,NaN,7523129
1,2003,1013.0,2136.0,111.056,33.0,37.0,verizonmmunications,verizonmmunications,NaN,7564776,adc telecommunications,adc telecommunications,NaN,7523129
2,2004,1013.0,2136.0,104.312,33.0,37.0,verizonmmunications,verizonmmunications,NaN,7564776,adc telecommunications,adc telecommunications,NaN,7523129
3,2005,1013.0,2136.0,146.000,33.0,37.0,verizonmmunications,verizonmmunications,NaN,7564776,adc telecommunications,adc telecommunications,NaN,7523129
4,2006,1013.0,2136.0,205.000,33.0,37.0,verizonmmunications,verizonmmunications,NaN,7564776,adc telecommunications,adc telecommunications,NaN,7523129


In [182]:
c_links['suppliers'] = 1
custExp = c_links[['year', 'customer_gvkey', 'salecs','suppliers']].groupby(['year','customer_gvkey']).sum().\
    reset_index().rename(columns = {'salecs': 'totalExp'})

custExp.head()



,year,customer_gvkey,totalExp,suppliers
0,2000,1038.0,38.220,2
1,2000,1045.0,38.093,4
2,2000,1078.0,5.070,2
3,2000,1121.0,7.883,1
4,2000,1177.0,284.677,1


In [183]:
print("Number of firms with no exp information and multiple suppliers: ", \
          sum(custExp[custExp.totalExp == 0].suppliers > 1))
print("Number of firms with no exp information and >5 suppliers: ", \
          sum(custExp[custExp.totalExp == 0].suppliers > 5))


Number of firms with no exp information and multiple suppliers:  210
Number of firms with no exp information and >5 suppliers:  6


Most of these firms have expenditure information. We can look at:
    - Expenditure-weighted (just do equal shares if no exp information)
    - Largest supplier

In [184]:
customerDB = c_links[['year','customer_gvkey','supplier_gvkey','salecs']].\
    merge(custExp).rename(columns = {'customer_gvkey': 'gvkey'}).drop_duplicates()
print(customerDB.shape)

customerDB.head()

(33849, 6)


,year,gvkey,supplier_gvkey,salecs,totalExp,suppliers
0,2002,2136.0,1013.0,111.056,892.202,13
1,2002,2136.0,3275.0,8.398,892.202,13
2,2002,2136.0,10286.0,16.987,892.202,13
3,2002,2136.0,10420.0,229.158,892.202,13
4,2002,2136.0,14340.0,9.432,892.202,13


Get the weather data.

In [190]:
g = pd.read_csv("../../data/companyData/weatherByEstablishment.csv").\
    drop(columns = {"Unnamed: 0"})

allWeather_withLags = pd.read_csv("../../data/companyData/allWeather_withLags.csv").\
    drop(columns = {"Unnamed: 0", 'yearQtr'})

averages = pd.read_csv("../../data/companyData/quarterlyStatsByZip.csv").\
    drop(columns = {"Unnamed: 0"}).rename(columns = {'ZIP': 'zipcode'})


averages['qtr'] = averages.quarter.str.slice(1,2).astype('float')
averages.drop(columns = {'quarter'}, inplace = True) 

averages.head()

allWeather_withLags       = allWeather_withLags.astype({'year':       'category',
                           'qtr':        'category',
                           'zipcode':    'category'})

averages                  = averages.astype({'qtr':        'category',
                           'zipcode':    'category'})

allWeather_byInd_withLags = pd.read_csv("../../data/companyData/allWeather_byInd_withLags.csv").\
    drop(columns = {"Unnamed: 0", 'yearQtr'})
allWeather_byInd_withLags = allWeather_byInd_withLags.astype({'year':       'category',
                           'qtr':        'category',
                           'zipcode':    'category'})

In [193]:
customersWithWeather = customers.merge(allWeather_withLags).merge(averages).merge(allWeather_byInd_withLags).merge(g)
print("customers: ", customersWithWeather.shape)

suppliersWithWeather = suppliers.merge(allWeather_withLags).merge(averages).merge(allWeather_byInd_withLags).merge(g)
print("supplier: ", suppliersWithWeather.shape)

suppliersWithWeather.to_csv("../../data/companyData/suppliersWithWeather.csv")
customersWithWeather.to_csv("../../data/companyData/customersWithWeather.csv")

'''suppliersWithWeather = pd.read_csv("../../data/companyData/suppliersWithWeather.csv").drop(columns = {'Unnamed: 0'})
customersWithWeather = pd.read_csv("../../data/companyData/customersWithWeather.csv").drop(columns = {'Unnamed: 0'})'''

frames = [customersWithWeather, suppliersWithWeather]

allCompanies = pd.concat(frames).drop_duplicates()

print(allCompanies.shape)

allCompanies.to_csv("../../data/companyData/allCompaniesWithWeather.csv")

customers:  (16644, 839)
supplier:  (27071, 839)
(38831, 839)


In [229]:
for col in customersWithWeather.columns:
    print(col)

gvkey
datadate
year
qtr
companyName
curcdq
assets
costGoodsSold
totalInv
netIncome
totalRevenue
cik
costat
add1
add2
addzip
city
sic
state
assetsLast
netIncomeLast
totalRevenueLast
costGoodsSoldLast
totalInvLast
incomeChange
revenueChange
costChange
inventoryChange
assetsPrev
assetsLagged
netIncomeLagged
roa_lagged
famafrench
earliestYear
ageTercile
sizeTercile
profitTercile
datacqtr
datafqtr
fyr
DATE
zipcode
employeesAtLocation
precip_annualquant_0.95
precip_annualquant_1xQtr
precip_annualquant_1xYr
precip_annualquant_1x5Qtrs
precip_annualquant_1x10Qtrs
precip_annualquant_1x5Yrs
precip_annualquant_1x10Yrs
precip_zipquant_0.95
precip_zipquant_1xQtr
precip_zipquant_1xYr
precip_zipquant_1x5Qtrs
precip_zipquant_1x10Qtrs
precip_zipquant_1x5Yrs
precip_zipquant_1x10Yrs
precip_zipQuarterquant_0.95
precip_zipQuarterquant_1xQtr
precip_zipQuarterquant_1xYr
precip_zipQuarterquant_1x5Qtrs
precip_zipQuarterquant_1x10Qtrs
precip_zipQuarterquant_1x5Yrs
precip_zipQuarterquant_1x10Yrs
temp_annualquant_

## Biggest Supplier
Focus on weather of biggest supplier.

First find the max by supplier. Add back in any rows with only 1 supplier.

In [194]:
customerDB.shape

(33849, 6)

In [195]:
# https://stackoverflow.com/questions/15705630/get-the-rows-which-have-the-max-value-in-groups-using-groupby
idx = customerDB.groupby(['year','gvkey']).salecs.\
    transform(max) == customerDB.salecs
largestSuppliers = customerDB[idx].reset_index(drop = True)
print(largestSuppliers.shape)

# find companies who only have one other supplier
singleSuppliers = customerDB[customerDB.suppliers == 1].reset_index(drop = True)
print(singleSuppliers.shape)

# find largest suppliers of different companies
largestSuppliers = largestSuppliers.append(singleSuppliers).drop_duplicates()
print(largestSuppliers.shape)



(7979, 6)
(5128, 6)
(9469, 6)


In [196]:
customerDB[['year','gvkey']][customerDB.year > 2009].drop_duplicates().shape

(5662, 2)

In [197]:
len(largestSuppliers.gvkey.unique())

1288

In [198]:
for column in suppliersWithWeather.columns:
    print(column)

gvkey
datadate
year
qtr
companyName
curcdq
assets
costGoodsSold
totalInv
netIncome
totalRevenue
cik
costat
add1
add2
addzip
city
sic
state
assetsLast
netIncomeLast
totalRevenueLast
costGoodsSoldLast
totalInvLast
incomeChange
revenueChange
costChange
inventoryChange
assetsPrev
assetsLagged
netIncomeLagged
roa_lagged
famafrench
earliestYear
ageTercile
sizeTercile
profitTercile
datacqtr
datafqtr
fyr
DATE
zipcode
employeesAtLocation
precip_annualquant_0.95
precip_annualquant_1xQtr
precip_annualquant_1xYr
precip_annualquant_1x5Qtrs
precip_annualquant_1x10Qtrs
precip_annualquant_1x5Yrs
precip_annualquant_1x10Yrs
precip_zipquant_0.95
precip_zipquant_1xQtr
precip_zipquant_1xYr
precip_zipquant_1x5Qtrs
precip_zipquant_1x10Qtrs
precip_zipquant_1x5Yrs
precip_zipquant_1x10Yrs
precip_zipQuarterquant_0.95
precip_zipQuarterquant_1xQtr
precip_zipQuarterquant_1xYr
precip_zipQuarterquant_1x5Qtrs
precip_zipQuarterquant_1x10Qtrs
precip_zipQuarterquant_1x5Yrs
precip_zipQuarterquant_1x10Yrs
temp_annualquant_

In [199]:
relevantVars = [x for x in suppliersWithWeather.columns if (('year' in x) | 
                                                 ('qtr' in x) |
                                                 ('gvkey' in x) |
                                                 ('_' in x)) & 
                                                ~('roa_lagged' in x) & 
                                                ~('yearQtr' in x)]

suppliers_toMerge = suppliersWithWeather[relevantVars]


for colname in suppliers_toMerge.columns[3:]:
    suppliers_toMerge.rename(columns = {colname: 'supplier_' + colname}, inplace = True)

    
suppliers_toMerge.rename(columns = {'gvkey': 'supplier_gvkey'},inplace = True)    

print(suppliers_toMerge.columns)


'''suppliers_toMerge = suppliersWithWeather[['year','qtr','gvkey','tmax_quant_1.0','precip_quant_1.0']].\
    rename(columns = {'gvkey': 'supplier_gvkey',
                      'tmax_quant_1.0': 'supplier_tmax_quant_1.0',
                      'precip_quant_1.0': 'supplier_precip_quant_1.0'})'''

Index(['supplier_gvkey', 'year', 'qtr', 'supplier_datacqtr',
       'supplier_datafqtr', 'supplier_precip_annualquant_0.95',
       'supplier_precip_annualquant_1xQtr', 'supplier_precip_annualquant_1xYr',
       'supplier_precip_annualquant_1x5Qtrs',
       'supplier_precip_annualquant_1x10Qtrs',
       ...
       'supplier_empWt_lag4_precip_zipQuarterquant_1x5Yrs',
       'supplier_empWt_lag4_temp_annualquant_0.95',
       'supplier_empWt_lag4_temp_annualquant_1x5Qtrs',
       'supplier_empWt_lag4_temp_annualquant_1x5Yrs',
       'supplier_empWt_lag4_temp_zipquant_0.95',
       'supplier_empWt_lag4_temp_zipquant_1x5Qtrs',
       'supplier_empWt_lag4_temp_zipquant_1x5Yrs',
       'supplier_empWt_lag4_temp_zipQuarterquant_0.95',
       'supplier_empWt_lag4_temp_zipQuarterquant_1x5Qtrs',
       'supplier_empWt_lag4_temp_zipQuarterquant_1x5Yrs'],
      dtype='object', length=801)


/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


"suppliers_toMerge = suppliersWithWeather[['year','qtr','gvkey','tmax_quant_1.0','precip_quant_1.0']].    rename(columns = {'gvkey': 'supplier_gvkey',\n                      'tmax_quant_1.0': 'supplier_tmax_quant_1.0',\n                      'precip_quant_1.0': 'supplier_precip_quant_1.0'})"

In [200]:
customersWithWeather.shape

(16644, 839)

In [201]:
len(set(supplierWtdAvgWeather.gvkey.unique()) - set(suppliers_toMerge.gvkey.unique()))


NameError: name 'supplierWtdAvgWeather' is not defined

In [202]:
largestSuppliers.head()

,year,gvkey,supplier_gvkey,salecs,totalExp,suppliers
0,2002,2136.0,10420.0,229.158,892.202,13
1,2003,2136.0,13440.0,214.000,1100.784,14
2,2004,2136.0,10420.0,332.586,1445.879,21
3,2005,2136.0,10420.0,508.518,2251.035,32
4,2006,2136.0,10420.0,551.124,2563.791,31


In [203]:
largestSuppliersWithWeather = customersWithWeather.merge(largestSuppliers[['year', 'gvkey', 'supplier_gvkey']]).merge(suppliers_toMerge)
largestSuppliersWithWeather.shape

(9682, 1638)

In [231]:
for col in largestSuppliersWithWeather.columns:
    print(col)

gvkey
datadate
year
qtr
companyName
curcdq
assets
costGoodsSold
totalInv
netIncome
totalRevenue
cik
costat
add1
add2
addzip
city
sic
state
assetsLast
netIncomeLast
totalRevenueLast
costGoodsSoldLast
totalInvLast
incomeChange
revenueChange
costChange
inventoryChange
assetsPrev
assetsLagged
netIncomeLagged
roa_lagged
famafrench
earliestYear
ageTercile
sizeTercile
profitTercile
datacqtr
datafqtr
fyr
DATE
zipcode
employeesAtLocation
precip_annualquant_0.95
precip_annualquant_1xQtr
precip_annualquant_1xYr
precip_annualquant_1x5Qtrs
precip_annualquant_1x10Qtrs
precip_annualquant_1x5Yrs
precip_annualquant_1x10Yrs
precip_zipquant_0.95
precip_zipquant_1xQtr
precip_zipquant_1xYr
precip_zipquant_1x5Qtrs
precip_zipquant_1x10Qtrs
precip_zipquant_1x5Yrs
precip_zipquant_1x10Yrs
precip_zipQuarterquant_0.95
precip_zipQuarterquant_1xQtr
precip_zipQuarterquant_1xYr
precip_zipQuarterquant_1x5Qtrs
precip_zipQuarterquant_1x10Qtrs
precip_zipQuarterquant_1x5Yrs
precip_zipQuarterquant_1x10Yrs
temp_annualquant_

In [204]:
largestSuppliersWithWeather.head()

,gvkey,datadate,year,qtr,companyName,curcdq,assets,costGoodsSold,totalInv,netIncome,...,supplier_empWt_lag4_precip_zipQuarterquant_1x5Yrs,supplier_empWt_lag4_temp_annualquant_0.95,supplier_empWt_lag4_temp_annualquant_1x5Qtrs,supplier_empWt_lag4_temp_annualquant_1x5Yrs,supplier_empWt_lag4_temp_zipquant_0.95,supplier_empWt_lag4_temp_zipquant_1x5Qtrs,supplier_empWt_lag4_temp_zipquant_1x5Yrs,supplier_empWt_lag4_temp_zipQuarterquant_0.95,supplier_empWt_lag4_temp_zipQuarterquant_1x5Qtrs,supplier_empWt_lag4_temp_zipQuarterquant_1x5Yrs
0,1045,20100331,2010,1,AMERICAN AIRLINES GROUP INC,USD,25525.0,4360.0,552.0,-505.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.000000,4.000000,0.0
1,1045,20100630,2010,2,AMERICAN AIRLINES GROUP INC,USD,25885.0,4575.0,569.0,-11.0,...,0.0,0.0,0.0,0.0,4.0,2.0,0.0,4.000000,2.000000,0.0
2,1045,20100930,2010,3,AMERICAN AIRLINES GROUP INC,USD,25357.0,4567.0,575.0,143.0,...,0.0,0.0,0.0,0.0,13.0,3.0,0.0,1.000000,0.000000,0.0
3,1045,20101231,2010,4,AMERICAN AIRLINES GROUP INC,USD,25088.0,4538.0,594.0,-98.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.000000,1.000000,0.0
4,1045,20140331,2014,1,AMERICAN AIRLINES GROUP INC,USD,43737.0,7382.0,1052.0,480.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.821918,1.780822,0.0


In [205]:
largestSuppliersWithWeather.to_csv("../../data/companyData/largestSuppliersWithWeather.csv")


In [211]:
for col in largestSuppliersWithWeather.columns:
    print(col)

gvkey
datadate
year
qtr
companyName
curcdq
assets
costGoodsSold
totalInv
netIncome
totalRevenue
cik
costat
add1
add2
addzip
city
sic
state
assetsLast
netIncomeLast
totalRevenueLast
costGoodsSoldLast
totalInvLast
incomeChange
revenueChange
costChange
inventoryChange
assetsPrev
assetsLagged
netIncomeLagged
roa_lagged
famafrench
earliestYear
ageTercile
sizeTercile
profitTercile
datacqtr
datafqtr
fyr
DATE
zipcode
employeesAtLocation
precip_annualquant_0.95
precip_annualquant_1xQtr
precip_annualquant_1xYr
precip_annualquant_1x5Qtrs
precip_annualquant_1x10Qtrs
precip_annualquant_1x5Yrs
precip_annualquant_1x10Yrs
precip_zipquant_0.95
precip_zipquant_1xQtr
precip_zipquant_1xYr
precip_zipquant_1x5Qtrs
precip_zipquant_1x10Qtrs
precip_zipquant_1x5Yrs
precip_zipquant_1x10Yrs
precip_zipQuarterquant_0.95
precip_zipQuarterquant_1xQtr
precip_zipQuarterquant_1xYr
precip_zipQuarterquant_1x5Qtrs
precip_zipQuarterquant_1x10Qtrs
precip_zipQuarterquant_1x5Yrs
precip_zipQuarterquant_1x10Yrs
temp_annualquant_

In [ ]:
largestSuppliersWithWeather = pd.read_csv("../../data/companyData/largestSuppliersWithWeather.csv")
largestSuppliersWithWeather.head()

In [ ]:
largestSuppliersWithWeather.columns[0:50]

## Sales-Weighted Average
If a company doesn't have sales-specific information, then assume equal shares. This doesn't happen for too many of the companies, thankfully.

In [212]:
customerDB = c_links[['year','customer_gvkey','supplier_gvkey','salecs']].\
    merge(custExp).rename(columns = {'customer_gvkey': 'gvkey'}).drop_duplicates()

customerDB['salesWeight'] = customerDB.salecs/customerDB.totalExp

customerDB.fillna(1, inplace = True)

Now merge this with the supplier weather data, and use the sales weights to find a sales-weighted average of the weather conditions for the suppliers.

In [213]:
relevantVars = [x for x in suppliersWithWeather.columns if (('year' in x) | 
                                                 ('qtr' in x) |
                                                 ('gvkey' in x) |
                                                 ('_' in x)) & 
                                                ~('roa_lagged' in x) & 
                                                ~('yearQtr' in x)]

suppliers_toMerge = suppliersWithWeather[relevantVars]


for colname in suppliers_toMerge.columns[3:]:
    suppliers_toMerge.rename(columns = {colname: 'supplier_' + colname}, inplace = True)

    
suppliers_toMerge.rename(columns = {'gvkey': 'supplier_gvkey'},inplace = True)    


/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [214]:
suppliers_toMerge.head()

,supplier_gvkey,year,qtr,supplier_datacqtr,supplier_datafqtr,supplier_precip_annualquant_0.95,supplier_precip_annualquant_1xQtr,supplier_precip_annualquant_1xYr,supplier_precip_annualquant_1x5Qtrs,supplier_precip_annualquant_1x10Qtrs,...,supplier_empWt_lag4_precip_zipQuarterquant_1x5Yrs,supplier_empWt_lag4_temp_annualquant_0.95,supplier_empWt_lag4_temp_annualquant_1x5Qtrs,supplier_empWt_lag4_temp_annualquant_1x5Yrs,supplier_empWt_lag4_temp_zipquant_0.95,supplier_empWt_lag4_temp_zipquant_1x5Qtrs,supplier_empWt_lag4_temp_zipquant_1x5Yrs,supplier_empWt_lag4_temp_zipQuarterquant_0.95,supplier_empWt_lag4_temp_zipQuarterquant_1x5Qtrs,supplier_empWt_lag4_temp_zipQuarterquant_1x5Yrs
0,1013,2010,1,2010Q1,2010Q2,2,0,0,0,0,...,0.332755,0.0,0.0,0.0,0.0,0.0,0.0,10.198698,5.252495,0.000000
1,66588,2010,1,2010Q1,2010Q2,2,0,0,0,0,...,0.420168,0.0,0.0,0.0,0.0,0.0,0.0,8.105042,3.739496,0.000000
2,113362,2010,1,2010Q1,2010Q1,2,0,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,7.831579,4.252632,0.000000
3,66588,2011,1,2011Q1,2011Q2,3,2,0,0,0,...,0.840336,0.0,0.0,0.0,0.0,0.0,0.0,8.008403,5.000000,0.525210
4,113362,2011,1,2011Q1,2011Q1,3,2,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,9.681973,5.634354,0.578231


For each of the supplier weather columns, multiply the variable by the fraction of sales attributable to that relationship.

In [217]:
supplierWeather = customerDB[['year','gvkey','supplier_gvkey','salesWeight']].merge(suppliers_toMerge)

for col in supplierWeather.columns[7:]:
        supplierWeather[col]   = supplierWeather.salesWeight*supplierWeather[col]
        
        

supplierWeather.drop(columns = {'supplier_gvkey','salesWeight'}, inplace = True)


print(supplierWeather.head())



# [['year','qtr','gvkey','supplier_tmax_quant_1.0','supplier_precip_quant_1.0']]

   year   gvkey qtr supplier_datacqtr supplier_datafqtr  \
0  2010  2136.0   1            2010Q1            2010Q2   
1  2010  2136.0   2            2010Q2            2010Q3   
2  2010  2136.0   3            2010Q3            2010Q4   
3  2010  9899.0   1            2010Q1            2010Q2   
4  2010  9899.0   2            2010Q2            2010Q3   

   supplier_precip_annualquant_0.95  supplier_precip_annualquant_1xQtr  \
0                          0.031038                           0.000000   
1                          0.186227                           0.062076   
2                          0.155189                           0.108633   
3                          0.127578                           0.000000   
4                          0.765471                           0.255157   

   supplier_precip_annualquant_1xYr  supplier_precip_annualquant_1x5Qtrs  \
0                          0.000000                                  0.0   
1                          0.000000             

In [218]:
supplierWtdAvgWeather = supplierWeather.groupby(['year','qtr','gvkey']).sum().reset_index().drop_duplicates()

In [219]:
supplierWtdAvgWeather.gvkey.unique()

array([  1045.,   1078.,   1161.,   1177.,   1240.,   1300.,   1327.,
         1380.,   1382.,   1440.,   1447.,   1487.,   1581.,   1602.,
         1663.,   1690.,   1704.,   1773.,   1919.,   1976.,   1995.,
         2019.,   2085.,   2086.,   2101.,   2136.,   2176.,   2184.,
         2220.,   2285.,   2290.,   2312.,   2403.,   2436.,   2504.,
         2556.,   2663.,   2751.,   2771.,   2783.,   2787.,   2817.,
         2968.,   2991.,   3036.,   3105.,   3121.,   3144.,   3203.,
         3226.,   3238.,   3243.,   3336.,   3413.,   3650.,   3708.,
         3734.,   3735.,   3813.,   3835.,   3851.,   3897.,   3905.,
         3918.,   3946.,   3964.,   3980.,   4016.,   4060.,   4066.,
         4093.,   4242.,   4279.,   4351.,   4383.,   4423.,   4430.,
         4503.,   4523.,   4560.,   4598.,   4601.,   4611.,   4839.,
         4843.,   4941.,   4990.,   5046.,   5047.,   5071.,   5073.,
         5109.,   5125.,   5234.,   5379.,   5439.,   5492.,   5523.,
         5606.,   56

Merge the supplier weighted average weather data with the customer data that has weather as well.

In [220]:
customersWithWeather.head()

,gvkey,datadate,year,qtr,companyName,curcdq,assets,costGoodsSold,totalInv,netIncome,...,empWt_lag4_precip_zipQuarterquant_1x5Yrs,empWt_lag4_temp_annualquant_0.95,empWt_lag4_temp_annualquant_1x5Qtrs,empWt_lag4_temp_annualquant_1x5Yrs,empWt_lag4_temp_zipquant_0.95,empWt_lag4_temp_zipquant_1x5Qtrs,empWt_lag4_temp_zipquant_1x5Yrs,empWt_lag4_temp_zipQuarterquant_0.95,empWt_lag4_temp_zipQuarterquant_1x5Qtrs,empWt_lag4_temp_zipQuarterquant_1x5Yrs
0,1045,20100331,2010,1,AMERICAN AIRLINES GROUP INC,USD,25525.0,4360.0,552.0,-505.0,...,0.408776,0.0,0.0,0.0,0.000000,0.000000,0.0,15.649236,7.754791,0.000858
1,1045,20110331,2011,1,AMERICAN AIRLINES GROUP INC,USD,27113.0,4758.0,595.0,-436.0,...,0.519124,0.0,0.0,0.0,0.000000,0.000000,0.0,2.064511,1.266357,0.005849
2,1045,20120331,2012,1,AMERICAN AIRLINES GROUP INC,USD,24511.0,5152.0,619.0,-1660.0,...,0.023659,0.0,0.0,0.0,0.006484,0.000000,0.0,13.109456,8.245812,0.029409
3,1045,20130331,2013,1,AMERICAN AIRLINES GROUP INC,USD,23852.0,4582.0,595.0,-341.0,...,0.010402,0.0,0.0,0.0,0.008462,0.002116,0.0,13.986355,7.595642,0.065795
4,1045,20140331,2014,1,AMERICAN AIRLINES GROUP INC,USD,43737.0,7382.0,1052.0,480.0,...,0.180929,0.0,0.0,0.0,0.000000,0.000000,0.0,5.650176,1.851254,0.157683


In [221]:
wtdAvgSuppliers = customersWithWeather.merge(supplierWtdAvgWeather)

wtdAvgSuppliers.head()

,gvkey,datadate,year,qtr,companyName,curcdq,assets,costGoodsSold,totalInv,netIncome,...,supplier_empWt_lag4_precip_zipQuarterquant_1x5Yrs,supplier_empWt_lag4_temp_annualquant_0.95,supplier_empWt_lag4_temp_annualquant_1x5Qtrs,supplier_empWt_lag4_temp_annualquant_1x5Yrs,supplier_empWt_lag4_temp_zipquant_0.95,supplier_empWt_lag4_temp_zipquant_1x5Qtrs,supplier_empWt_lag4_temp_zipquant_1x5Yrs,supplier_empWt_lag4_temp_zipQuarterquant_0.95,supplier_empWt_lag4_temp_zipQuarterquant_1x5Qtrs,supplier_empWt_lag4_temp_zipQuarterquant_1x5Yrs
0,1045,20100331,2010,1,AMERICAN AIRLINES GROUP INC,USD,25525.0,4360.0,552.0,-505.0,...,0.901163,0.0,0.0,0.0,0.0,0.0,0.0,17.352990,8.394518,0.000000
1,1045,20110331,2011,1,AMERICAN AIRLINES GROUP INC,USD,27113.0,4758.0,595.0,-436.0,...,0.811808,0.0,0.0,0.0,0.0,0.0,0.0,1.940037,0.371771,0.018450
2,1045,20120331,2012,1,AMERICAN AIRLINES GROUP INC,USD,24511.0,5152.0,619.0,-1660.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,1045,20130331,2013,1,AMERICAN AIRLINES GROUP INC,USD,23852.0,4582.0,595.0,-341.0,...,0.410918,0.0,0.0,0.0,0.0,0.0,0.0,17.748535,9.212420,0.040792
4,1045,20140331,2014,1,AMERICAN AIRLINES GROUP INC,USD,43737.0,7382.0,1052.0,480.0,...,0.045628,0.0,0.0,0.0,0.0,0.0,0.0,11.898162,4.684855,0.000000


In [222]:
wtdAvgSuppliers.shape

(11612, 1635)

In [223]:
wtdAvgSuppliers.to_csv("../../data/companyData/wtdAvgSuppliers.csv")

In [224]:
wtdAvgSuppliers.head()

,gvkey,datadate,year,qtr,companyName,curcdq,assets,costGoodsSold,totalInv,netIncome,...,supplier_empWt_lag4_precip_zipQuarterquant_1x5Yrs,supplier_empWt_lag4_temp_annualquant_0.95,supplier_empWt_lag4_temp_annualquant_1x5Qtrs,supplier_empWt_lag4_temp_annualquant_1x5Yrs,supplier_empWt_lag4_temp_zipquant_0.95,supplier_empWt_lag4_temp_zipquant_1x5Qtrs,supplier_empWt_lag4_temp_zipquant_1x5Yrs,supplier_empWt_lag4_temp_zipQuarterquant_0.95,supplier_empWt_lag4_temp_zipQuarterquant_1x5Qtrs,supplier_empWt_lag4_temp_zipQuarterquant_1x5Yrs
0,1045,20100331,2010,1,AMERICAN AIRLINES GROUP INC,USD,25525.0,4360.0,552.0,-505.0,...,0.901163,0.0,0.0,0.0,0.0,0.0,0.0,17.352990,8.394518,0.000000
1,1045,20110331,2011,1,AMERICAN AIRLINES GROUP INC,USD,27113.0,4758.0,595.0,-436.0,...,0.811808,0.0,0.0,0.0,0.0,0.0,0.0,1.940037,0.371771,0.018450
2,1045,20120331,2012,1,AMERICAN AIRLINES GROUP INC,USD,24511.0,5152.0,619.0,-1660.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,1045,20130331,2013,1,AMERICAN AIRLINES GROUP INC,USD,23852.0,4582.0,595.0,-341.0,...,0.410918,0.0,0.0,0.0,0.0,0.0,0.0,17.748535,9.212420,0.040792
4,1045,20140331,2014,1,AMERICAN AIRLINES GROUP INC,USD,43737.0,7382.0,1052.0,480.0,...,0.045628,0.0,0.0,0.0,0.0,0.0,0.0,11.898162,4.684855,0.000000


In [225]:
wtdAvgSuppliers.columns[wtdAvgSuppliers.columns.str.contains('Tercile')]

Index(['ageTercile', 'sizeTercile', 'profitTercile'], dtype='object')